In [ ]:
import numpy as np
import xarray as xr

This notebook will show the advantage of using `xr.apply_ufunc` over directly calling the functions.
It will treat three options: vectorize, output_core_dims and input_core_dims.

## vectorize

Let's start where we left off the last notebook.

In [ ]:
def kinetic_energy_J(m_kg, v_mps):
    return 0.5 * m_kg * v_mps**2

In [ ]:
ds = xr.Dataset()
ds["v_mps"] = np.linspace(0, 10)
ds.v_mps.attrs = {"long_name": "Velocity", "units": "m/s"}
ds["m_kg"] = np.linspace(3, 17)
ds.m_kg.attrs = {"long_name": "Mass", "units": "kg"}
ds["Ekin_j"] = kinetic_energy_J(ds.m_kg, ds.v_mps)
ds.Ekin_j.attrs = {"long_name": "Energy", "units": "J"}
ds

Say we introduce a check for the mass, not allowing negative masses.

In [ ]:
def kinetic_energy_J_check(m_kg, v_mps):
    if m_kg < 0:  # Not allowed for numpy array
        raise ValueError("Mass should be positive!")
    return 0.5 * m_kg * v_mps**2

This will raise an error as the function can only handle `m_kg` of type `float` and `int`, not an entire `numpy` array.

In [ ]:
ds = xr.Dataset()
ds["v_mps"] = np.linspace(0, 10)
ds.v_mps.attrs = {"long_name": "Velocity", "units": "m/s"}
ds["m_kg"] = np.linspace(3, 17)
ds.m_kg.attrs = {"long_name": "Mass", "units": "kg"}
ds["Ekin_J"] = kinetic_energy_J_check(ds.m_kg, ds.v_mps)
ds.Ekin_J.attrs = {"long_name": "Energy", "units": "J"}
ds

Using `vectorize=True` in `xr.apply_ufunc` will fix this as it will deconstruct the input arrays and call the function for each value seperately. This is much slower than before, but sometimes functions can't handle inputs that are arrays, but expect `int` or `float`. A good example are `qutip` simulations as we will see later.

In [ ]:
ds = xr.Dataset()
ds["v_mps"] = np.linspace(0, 10)
ds.v_mps.attrs = {"long_name": "Velocity", "units": "m/s"}
ds["m_kg"] = np.linspace(3, 17)
ds.m_kg.attrs = {"long_name": "Mass", "units": "kg"}
ds["Ekin_j"] = xr.apply_ufunc(kinetic_energy_J_check, ds.m_kg, ds.v_mps, vectorize=True)
ds.Ekin_j.attrs = {"long_name": "Energy", "units": "J"}
ds

This works! However, what we should have done:

In [ ]:
def kinetic_energy_j_check_fast(m_kg, v_mps):
    if np.all(m_kg < 0):  # Allowed for numpy array
        raise ValueError("Mass should be positive!")
    return 0.5 * m_kg * v_mps**2


ds = xr.Dataset()
ds["v_mps"] = np.linspace(0, 10)
ds.v_mps.attrs = {"long_name": "Velocity", "units": "m/s"}
ds["m_kg"] = np.linspace(3, 17)
ds.m_kg.attrs = {"long_name": "Mass", "units": "kg"}
ds["Ekin_J"] = kinetic_energy_j_check_fast(ds.m_kg, ds.v_mps)
ds.Ekin_J.attrs = {"long_name": "Energy", "units": "J"}
ds

Use vectorize only if you cannot achieve the same thing with a native numpy function. This will be so much faster.

## output_core_dims 1

Now we will show another advantage of `xr.apply_ufunc`: handeling functions that do not have the same data format for the inputs as for the outputs.

We now want to draw a random sample from a normal distribution:

In [ ]:
mean = 0
sigma = 1
points = 10
random = np.random.normal(mean, sigma, points)
random

In [ ]:
mean = 0
points = 10
ds = xr.Dataset()
ds["sigma"] = 1
ds["random"] = np.random.normal(mean, ds.sigma, points)
ds

If we now want to sweep the sigma of the distribution, this will give problems:

In [ ]:
mean = 0
points = 10
ds = xr.Dataset()
ds["sigma"] = np.linspace(1, 2)
ds["random"] = np.random.normal(mean, ds.sigma, points)
ds

The reason why this fails is that `np.random.normal` does only accepts floats as inputs so we should vectorize:

In [ ]:
mean = 0
points = 10
ds = xr.Dataset()
ds["sigma"] = np.linspace(1, 2)
ds["random"] = xr.apply_ufunc(np.random.normal, mean, ds.sigma, points, vectorize=True)
ds

The reason why it still fails is that we need to tell `xr.apply_ufunc` that it receives an array back while it only supplied a `float`. We can do this by using `output_core_dims=[['random']]`. It is a list with the number of return values and each element is a list with the dimension that are coming back extra.

In [ ]:
mean = 0
points = 10
ds = xr.Dataset()
ds["sigma"] = np.linspace(1, 2)
ds["random"] = xr.apply_ufunc(
    np.random.normal,
    mean,
    ds.sigma,
    points,
    output_core_dims=[["random"]],
    vectorize=True,
)
ds

## output_core_dims 2

Another example of using `output_core_dims` is when we have multiple return values:

In [ ]:
def kinetic_energy_and_momentum(m_kg, v_m_per_s):
    energy = 0.5 * m_kg * v_m_per_s**2
    momentum = m_kg * v_m_per_s
    return energy, momentum

Now we need to say that there are two return values, both without additional dims, so `output_core_dims=[[], []]`

In [ ]:
ds = xr.Dataset()
ds["m"] = 5
ds["v"] = np.linspace(0, 10)
ds["Ekin"], ds["momentum"] = xr.apply_ufunc(
    kinetic_energy_and_momentum, ds.m, ds.v, output_core_dims=[[], []]
)
ds

## input_core_dims

Let's now compute the standard deviation of this random sample:

In [ ]:
mean = 0
sigma = 1
points = 10
random = np.random.normal(mean, sigma, points)
std = np.std(random)
std

With the `xarray` implementation:

In [ ]:
mean = 0
points = 10
ds = xr.Dataset()
ds["sigma"] = np.linspace(1, 2)
ds["random"] = xr.apply_ufunc(
    np.random.normal,
    mean,
    ds.sigma,
    points,
    output_core_dims=[["random"]],
    vectorize=True,
)
ds["stderr"] = xr.apply_ufunc(np.std, ds.random)
ds

Which fails because it expected the function to return an array of the size of the random dimension.

So we need to tell `xr.apply_ufunc` that the function "eats" an dimension, we do this by setting `input_core_dims`=[['random']], telling it that the first (and in this case only) argument of the function should be of the same shape as random and that this dimension will not be part of the return values.

In [ ]:
mean = 0
points = 10
ds = xr.Dataset()
ds["sigma"] = np.linspace(1, 2)
ds["random"] = xr.apply_ufunc(
    np.random.normal,
    mean,
    ds.sigma,
    points,
    output_core_dims=[["random"]],
    vectorize=True,
)
ds["stderr"] = xr.apply_ufunc(np.std, ds.random, input_core_dims=[["random"]])
ds

Let's now sweep the sigma of the distribution again:

In [ ]:
mean = 0
points = 10
ds = xr.Dataset()
ds["sigma"] = np.linspace(1, 2)
ds["random"] = xr.apply_ufunc(
    np.random.normal,
    mean,
    ds.sigma,
    points,
    output_core_dims=[["random"]],
    vectorize=True,
)
ds["stderr"] = xr.apply_ufunc(np.std, ds.random, input_core_dims=[["random"]])
ds

Now the problem is that mean has collapsed the whole input array to a single float including the sigma dimension. We need to tell the function that it needs run the function separately for each sigma input with `vectorize=True`.

In [ ]:
mean = 0
points = 10
ds = xr.Dataset()
ds["sigma"] = np.linspace(1, 2)
ds["random"] = xr.apply_ufunc(
    np.random.normal,
    mean,
    ds.sigma,
    points,
    output_core_dims=[["random"]],
    vectorize=True,
)
ds["stderr"] = xr.apply_ufunc(
    np.std, ds.random, input_core_dims=[["random"]], vectorize=True
)
ds

In [ ]:
from matplotlib import pyplot as plt

ds.sigma.plot()
ds.stderr.plot.scatter()
plt.show()